<a href="https://colab.research.google.com/github/pulipulichen/colab-File-Browser/blob/main/colab/colab-File-Browser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **colab-File-Browser**

@TODO

1. 按 `Runtime` > `Run all`，或按Ctrl + F9，直接執行腳本即可。
2. 若遇到 Restart...

@TODO
操作說明請看：https://ppt.cc/fqbnwx

# Script

## Configurations for script

In [1]:
PROJECT_OWNER="pulipulichen"
PROJECT_NAME="colab-File-Browser"


測試的方式

1. `Runtime` > `Run all` (Ctrl + F9)
2. 等待Cloudflare網址可連

## Core Tools

### curl_local_url()

In [2]:
import os
import time
import subprocess

inited = True
if os.path.isfile('.inited') is False:
  inited = False

def curl_local_url(port):
  while True:
    try:
      output = subprocess.check_output(['curl', '-s', 'http://127.0.0.1:' + str(port) + '/']).decode('utf-8')
      print("Success to connect http://127.0.0.1:" + str(port))
      break  # Break out of the loop if successful
    except subprocess.CalledProcessError:
      # print("Check http://127.0.0.1:" + str(port) + "/ failed. Retrying in 5 seconds...")
      time.sleep(5)
      continue  # Retry in case of failure

### wait_for_cloudflare()

In [3]:
import os
import time

def wait_for_cloudflare():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.cloudflare.url'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")


### wait_for_docker_web_ready()

In [4]:
import os
import time

def wait_for_docker_web_ready():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.docker-web.ready'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")

  time.sleep(10)

### keep_waiting()

In [5]:
import time

def keep_waiting():
  while True:
    time.sleep(1)  # Check every 1 second


### mountGDrive()

In [6]:
import os
from subprocess import getoutput
from google.colab import drive
import subprocess

def mountGDrive():
  if not os.path.exists("/google-drive"):
    drive.mount('/google-drive')  # access drive
  folder_path = "/google-drive/MyDrive/docker-app/" + PROJECT_NAME + "/"
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  target_folder_path = "/content/docker-app/"
  if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

  command = ["ln", "-s", folder_path, target_folder_path]
  subprocess.run(command, check=True)

### clearDockerData()

In [7]:
import os
from subprocess import getoutput
import subprocess

def clearDockerData():
  stop_udocker()
  command = ["rm", "-rf", "/content/docker-app/" + PROJECT_NAME]
  subprocess.run(command, check=True)


### get_config_from_yaml()

In [8]:
import yaml
import subprocess
import cherrypy
import time

if inited is False:
  !pip install CherryPy==18.8.0
  !pip install udocker==1.3.10
  !udocker --allow-root install

  from google.colab.output import eval_js
  !rm -rf ./sample_data*

  !pip install pyyaml==6.0.1

def get_config_from_yaml():
  yaml_file_path = f"./.docker-app/{PROJECT_NAME}/app-build/docker-compose-template.yml"

  with open(yaml_file_path, "r") as file:
    yaml_data = yaml.safe_load(file)
  # print(yaml_data)

  # Assuming there is only one service named "app" in your YAML
  webapp_ports = yaml_data.get("services", {}).get("app", {}).get("ports", [])
  WEBAPP_PORT = None

  for port_mapping in webapp_ports:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(port_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_port, container_port = port_mapping.split(":")
        WEBAPP_PORT = host_port
        break

  # ============
  environments = yaml_data.get("services", {}).get("app", {}).get("environment", [])
  LOCAL_VOLUMN_PATH = get_environments_from_yaml(environments, 'LOCAL_VOLUMN_PATH')
  RUN_COMMAND = get_environments_from_yaml(environments, 'RUN_COMMAND')

  # ============
  app = yaml_data.get("services", {}).get("app", {})
  # print(app)
  IMAGE_NAME = app['image']

  return {
    "WEBAPP_PORT": WEBAPP_PORT,
    "LOCAL_VOLUMN_PATH": LOCAL_VOLUMN_PATH,
    "IMAGE_NAME": IMAGE_NAME,
    "RUN_COMMAND": RUN_COMMAND
  }


#### get_environments_from_yaml()

In [9]:
def get_environments_from_yaml(environments, key, default_value = None):

  for env_mapping in environments:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(env_mapping, str) and key + "=" in env_mapping:
        # print(port_mapping)
        key, value = env_mapping.split("=")
        default_value = value
        break

  return default_value

## udocker

### stop_udocker()

In [10]:
def stop_udocker():
  #!udocker --allow-root ps
  !udocker --allow-root ps | awk 'NR > 1 {print $1}' | xargs -I {} udocker --allow-root rm {}
  !pgrep java && pkill java && sleep 5


### run_udocker()

In [11]:
def run_udocker():
  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  # !rm -f /content/docker-app/$PROJECT_NAME/.docker-web.ready || true
  # !rm -f /content/docker-app/$PROJECT_NAME/.cloudflare.url || true
  ![ -e /content/docker-app/$PROJECT_NAME/.cloudflare.url ] && rm /content/docker-app/$PROJECT_NAME/.cloudflare.url
  ![ -e /content/docker-app/$PROJECT_NAME/.docker-web.ready ] && rm /content/docker-app/$PROJECT_NAME/.docker-web.ready

  RUN_IN_BACKGROUND = True
  if YAML_CONFIG["WEBAPP_PORT"] is None:
    RUN_IN_BACKGROUND = False

  # For testing purposes
  RUN_IN_BACKGROUND = False

  if RUN_IN_BACKGROUND is False:
    run_udocker_in_foreground()
  else:
    run_udocker_in_background()


### run_udocker_in_foreground()

In [12]:
def run_udocker_in_foreground():

  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  # print('Run container in foreground..')
  # subprocess.run(['udocker', '--allow-root', 'run', '-p' ,f'"{WEBAPP_PORT}:{WEBAPP_PORT}"', f'--volume="/content/docker-app/{PROJECT_NAME}:{LOCAL_VOLUMN_PATH}"', IMAGE_NAME])
  # !udocker --allow-root run -p "$WEBAPP_PORT:$WEBAPP_PORT" --volume="/content/docker-app/$PROJECT_NAME:$LOCAL_VOLUMN_PATH" $IMAGE_NAME

  # print(LOCAL_VOLUMN_PATH)
  # print(RUN_COMMAND)

  !rm -rf /content/docker-app/$PROJECT_NAME
  !mkdir -p /content/docker-app/$PROJECT_NAME

  udocker_command = get_udocker_run_command()
  !$udocker_command

  print("Finish")

### run_udocker_in_background()

In [13]:
def run_udocker_in_background():

  if os.path.isfile('.cloudflared') is False:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O .cloudflared
    !chmod a+x .cloudflared

  stop_udocker()

  !mkdir -p /content/docker-app/$PROJECT_NAME

  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  !rm -rf ./*nohup.out
  !rm -rf ./docker-app/$PROJECT_NAME/.cloudflare.url

  print('Run container in background..')
  ![ -e /content/.docker-app/udocker.sh ] && rm /content/.docker-app/udocker.sh
  udocker_command = get_udocker_run_command()
  !echo "nohup $udocker_command > .nohup.out 2>&1 &" >> /content/.docker-app/udocker.sh
  !chmod +x /content/.docker-app/udocker.sh
  !bash /content/.docker-app/udocker.sh

  cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port' : int(YAML_CONFIG["WEBAPP_PORT"])})
  # wait_for_cloudflare()
  curl_local_url(YAML_CONFIG["WEBAPP_PORT"])
  wait_for_docker_web_ready()

  print('============================================================================')
  print('請找到下面訊息的網址。網址的結尾必須為「.trycloudflare.com 」。')
  print('例如：https://organised-norton-talks-autos.trycloudflare.com')
  print('')
  print('如果超過5分鐘仍然沒有顯示資訊，請中斷執行後，重新執行這個cell。')
  print('============================================================================')
  #!cat /content/docker-app/docker-web-Apache-Solr/solrconfig.xml | grep dc.publisher
  !./.cloudflared --url "http://127.0.0.1:$YAML_CONFIG["WEBAPP_PORT"]"


### get_udocker_run_command()

In [14]:
def get_udocker_run_command():
  YAML_CONFIG = get_config_from_yaml()

  command = "udocker --allow-root run " + \
    "-p " + YAML_CONFIG["WEBAPP_PORT"] + ":" + YAML_CONFIG["WEBAPP_PORT"] + \
    " " + \
    "--volume=/content/docker-app/" + PROJECT_NAME + ":" + YAML_CONFIG["LOCAL_VOLUMN_PATH"] + \
    " " + \
    YAML_CONFIG["IMAGE_NAME"]

  print(command)

  return command

## main() (must be the last)

In [15]:

if inited is False:
  %cd /content
  !touch .inited
  inited = True

# ==================

!mkdir -p ./.docker-app

if os.path.isdir('/content/.docker-app/' + PROJECT_NAME) is False:
  %cd /content/.docker-app
  !git clone "https://github.com/{PROJECT_OWNER}/{PROJECT_NAME}.git"
else:
  %cd /content/.docker-app/$PROJECT_NAME
  !git reset --hard
  !git pull --force
%cd /content/

# ==================

def main():
  run_udocker()

/content/.docker-app/colab-File-Browser
HEAD is now at 1127d3c 西元2025年02月02日 (週日) 22時06分03秒 CST
Already up to date.
/content


In [16]:
# For Testing purposes

#clearDockerData()

# **Runtime**

In [17]:
# mountGDrive() # 開啟Google Drive掛載功能
main()

udocker --allow-root run -p 8081:8081 --volume=/content/docker-app/colab-File-Browser:/data pudding/docker-web:colab-file-browser-20250202.220343
Error: create container: getting layers or json
Info: downloading layer sha256:a26af6de5f791e11db1b79daa044d53d5dacb458925aec53b2da8976e6818f10
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:22a000724d336b2928d092eb5c6706338a1ec497f1c65457557fefb5d8a3a79b
Info: downloading layer sha256:765ac7573f365d1e418b43cdb31f052323c85faa2a0dd0ddc4d5c1fbcfe726bd
Info: downloading layer sha256:ba3cfb2a229676c3f7eb5f6dadb75a0c08f93a748824e78afbcabf62c1fdd5ff
Info: downloading layer sha256:118df6eacbe30c327bac9be3b223aaccbcfe10a43285692411a4742295cbfa8a
Info: downloading layer sha256:4c4be17cfc167ba19a8e2e30ea1ea3ef3af27f923ae7f6899d46ac0ebef72170
Info: downloading layer sha256:c7ee4fb04a31b86868cbac801da6f9deb9d9b6c4306a60f590b5da99196b8a52
Info: downloading layer sha256:d05adeb